# pandas的汇总操作

pandas的拼接分为两种：
- 级联：pd.concat, pd.append
- 合并：pd.merge, pd.join

## 0. 回顾numpy的级联

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
#1. numpy级联维度上的形状要保持一致
arr1 = np.ones(shape=(2,3))
arr2 = np.zeros(shape=(4,3))

In [3]:
display(arr1, arr2)

array([[1., 1., 1.],
       [1., 1., 1.]])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [6]:
np.concatenate((arr1, arr2), axis=1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

============================================

练习12：

1. 生成2个3*3的矩阵，对其分别进行两个维度上的级联

============================================

为方便讲解，我们首先定义一个生成DataFrame的函数：

## 1. 使用pd.concat()级联

In [ ]:
#1. 级联方向上的形状不同，同样可以连接
#2. 以索引对齐的方式连接，所以跟顺序没关系

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
join='outer'
join_axes=None
ignore_index=False
keys
```

### 1)  简单级联

和np.concatenate一样，优先增加行数（默认axis=0）

In [13]:
score1 = DataFrame(data=np.random.randint(0,100,size=(3,3)), columns=["python","java","php"], index=list("ABC"))
score2 = DataFrame(data=np.random.randint(0,100,size=(4,3)), columns=["java","python","php"], index=list("DBHC"))
display(score1, score2)

,python,java,php
A,61,17,33
B,98,0,47
C,26,83,69


,java,python,php
D,96,63,84
B,86,83,66
H,92,11,28
C,46,25,30


In [10]:
# objs 要参与级联的pandas对象的列表、元组
# 默认是纵向连接
pd.concat(objs=(score1, score2))

,java,php,python
A,56,35,99
B,44,16,54
C,85,90,69
D,9,40,22
F,86,78,93
H,42,27,90
K,85,57,32


可以通过设置axis来改变级联方向

In [14]:
# 列： 字段
# 行： 数据项
pd.concat(objs=(score1, score2), axis=1)

,python,java,php,java,python,php
A,61.0,17.0,33.0,NaN,NaN,NaN
B,98.0,0.0,47.0,86.0,83.0,66.0
C,26.0,83.0,69.0,46.0,25.0,30.0
D,NaN,NaN,NaN,96.0,63.0,84.0
H,NaN,NaN,NaN,92.0,11.0,28.0


级连会把该方向上索引相同的元素放在一行（一列），index/columns在级联时可以重复

也可以选择忽略ignore_index，重新索引

In [15]:
columns = ["成本","费用","销售额"]
order1 = DataFrame(data=np.random.randint(0,100,size=(5,3)), columns=columns)
order2 = DataFrame(data=np.random.randint(0,100,size=(8,3)), columns=columns)
display(order1, order2)

,成本,费用,销售额
0,8,71,78
1,53,49,72
2,84,66,57
3,96,28,12
4,59,55,18


,成本,费用,销售额
0,7,30,57
1,2,60,64
2,88,11,18
3,90,97,39
4,29,36,72
5,69,3,25
6,54,72,11
7,15,18,19


In [27]:
# 原始表的索引没有实际意义，可以忽略处理，避免重复
order = pd.concat((order1, order2), ignore_index=True)
order

,成本,费用,销售额
0,8,71,78
1,53,49,72
2,84,66,57
3,96,28,12
4,59,55,18
5,7,30,57
6,2,60,64
7,88,11,18
8,90,97,39
9,29,36,72


In [26]:
# 2. 如果级联后，原始索引不能忽略，但又希望避免重复，可以做成多层级表格
pd.concat((order1, order2), axis=1, keys=["第一季度","第二季度"]).mean()

第一季度  成本     60.000
      费用     53.800
      销售额    47.400
第二季度  成本     44.250
      费用     40.875
      销售额    38.125
dtype: float64

或者使用多层索引 keys  

concat([x,y],keys=['x','y'])

============================================

练习13：

1. 想一想级联的应用场景？

2. 使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd

3. 假设新增考试学科"计算机"，如何实现？

4. 新增王老五同学的成绩，如何实现？

【使用级联来实现】

============================================

In [ ]:
# 1想一想级联的应用场景？
# 数据表结构相似或相同的汇总操作

In [29]:
# 2使用昨天的知识，建立一个期中考试张三、李四的成绩表ddd
score = DataFrame(data=np.random.randint(0,100,size=(2,3)), columns=["语文","数学","英语"], index=["张三","李四"])
score

,语文,数学,英语
张三,98,63,34
李四,67,7,80


In [30]:
s1 = DataFrame(data=np.random.randint(0,100,size=(2,1)), columns=["计算机"], index=["张三","李四"])
s1

,计算机
张三,79
李四,71


In [31]:
# 假设新增考试学科"计算机"，如何实现？
score = pd.concat((score, s1), axis=1)

In [32]:
score

,语文,数学,英语,计算机
张三,98,63,34,79
李四,67,7,80,71


In [33]:
s2 = DataFrame(data=np.random.randint(0,100,size=(1,4)), index=["王老五"], columns=score.columns)
s2

,语文,数学,英语,计算机
王老五,94,43,89,49


In [34]:
score = pd.concat((score, s2))
score

,语文,数学,英语,计算机
张三,98,63,34,79
李四,67,7,80,71
王老五,94,43,89,49


### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

有3种连接方式：

- 外连接：补NaN（默认模式）

- 内连接：只连接匹配的项

- 连接指定轴 join_axes

In [36]:
df1 = DataFrame(data=np.random.randint(0,100,size=(3,5)), columns=list("ABCDE"))
df2 = DataFrame(data=np.random.randint(-100,0,size=(3,6)), columns=list("BCDEFG"))
display(df1, df2)

,A,B,C,D,E
0,19,90,21,82,75
1,85,57,67,33,44
2,91,11,98,77,29


,B,C,D,E,F,G
0,-84,-60,-50,-44,-50,-20
1,-72,-57,-29,-78,-34,-45
2,-31,-4,-18,-2,-74,-63


In [37]:
# outer连接，外连接,保存连接的表的所有字段，缺失值补空值
pd.concat((df1, df2), join='outer')

,A,B,C,D,E,F,G
0,19.0,90,21,82,75,NaN,NaN
1,85.0,57,67,33,44,NaN,NaN
2,91.0,11,98,77,29,NaN,NaN
0,NaN,-84,-60,-50,-44,-50.0,-20.0
1,NaN,-72,-57,-29,-78,-34.0,-45.0
2,NaN,-31,-4,-18,-2,-74.0,-63.0


In [38]:
# inner连接, 内连接，只保留公共字段
# inner连接会导致数据丢失
pd.concat((df1, df2), join='inner')

,B,C,D,E
0,90,21,82,75
1,57,67,33,44
2,11,98,77,29
0,-84,-60,-50,-44
1,-72,-57,-29,-78
2,-31,-4,-18,-2


In [43]:
# 只保留特定的几个字段
pd.concat((df1, df2), join_axes=[pd.Index(["D","E"])])

,D,E
0,82,75
1,33,44
2,77,29
0,-50,-44
1,-29,-78
2,-18,-2


In [46]:
# 左连接、右连接（两张表连接）
pd.concat((df1, df2), join_axes=[df1.columns])

,A,B,C,D,E
0,19.0,90,21,82,75
1,85.0,57,67,33,44
2,91.0,11,98,77,29
0,NaN,-84,-60,-50,-44
1,NaN,-72,-57,-29,-78
2,NaN,-31,-4,-18,-2


============================================

练习14：

    假设【期末】考试ddd2的成绩没有张三的，只有李四、王老五、赵小六的，使用多种方法级联

============================================

### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

注意:append函数只是沿着axis=0的方向进行级联

In [47]:
df1.append(df2)

,A,B,C,D,E,F,G
0,19.0,90,21,82,75,NaN,NaN
1,85.0,57,67,33,44,NaN,NaN
2,91.0,11,98,77,29,NaN,NaN
0,NaN,-84,-60,-50,-44,-50.0,-20.0
1,NaN,-72,-57,-29,-78,-34.0,-45.0
2,NaN,-31,-4,-18,-2,-74.0,-63.0


============================================

练习15：

    新建一个只有张三李四王老五的期末考试成绩单ddd3，使用append()与期中考试成绩表ddd级联

============================================

In [ ]:
小结：
使用场景：
pd.concat()
    objs: (df1,df2..dfn) 
    axis: axis=0, 1
    join: inner outer(default)
    ignore_index:  忽略原始索引
    keys:  设置多层级索引，保留原始索引
    join_axes: 指定特定的连接标签

## 2. 使用pd.merge()合并

In [ ]:
# 合并只跟列有关，两张表要去找内容相同的列来进行合并
# 合并的两张表一定存在至少一列，在内容上有对应关系，至少是一对一、一对多、多对多中一种
# 合并可以同时参考多列进行，这取决于你的业务需求

merge与concat的区别在于，merge需要依据某一共同【列】来进行合并

使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

注意每一列元素的顺序不要求一致

In [51]:
table1 = pd.read_excel('关系表.xlsx',sheet_name=1)
table2 = pd.read_excel('关系表.xlsx',sheet_name=2)
table3 = pd.read_excel('关系表.xlsx',sheet_name=3)
table4 = pd.read_excel('关系表.xlsx',sheet_name=4)

In [52]:
table1

,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


In [53]:
table2

,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


In [54]:
table3

,经销商,发货地区,手机型号
0,dancer,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


In [55]:
table4

,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


#### 1) 一对一合并

In [56]:
pd.merge(table1, table2)

,手机型号,参考价格,重量
0,windowsPhone,2500,0.50
1,iPhone,7500,0.40
2,Android,4000,0.45


In [57]:
pd.concat((table1, table2))

,参考价格,手机型号,重量
0,2500.0,windowsPhone,NaN
1,7500.0,iPhone,NaN
2,4000.0,Android,NaN
0,NaN,windowsPhone,0.50
1,NaN,iPhone,0.40
2,NaN,Android,0.45
3,NaN,other,0.60


#### 2) 多对一合并

In [58]:
table3

,经销商,发货地区,手机型号
0,dancer,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


In [59]:
table1

,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


In [60]:
pd.merge(table1, table3)

,手机型号,参考价格,经销商,发货地区
0,windowsPhone,2500,petter,shenzhen
1,iPhone,7500,dancer,beijing
2,iPhone,7500,tom,guangzhou
3,Android,4000,lucy,beijing
4,Android,4000,mery,guangzhou


In [61]:
table4

,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


#### 3) 多对多合并

In [62]:
# 1. 默认把字段名字（列标签）相同的列作为合并的依据
# 2. 默认如果有多个列的标签相同，则会同时参考多列合并
pd.merge(table3, table4)

,经销商,发货地区,手机型号,价格
0,dancer,beijing,iPhone,7000
1,lucy,beijing,Android,3600
2,tom,guangzhou,iPhone,7600
3,petter,shenzhen,windowsPhone,2750
4,mery,guangzhou,Android,4200


#### 4) key的规范化

- 使用on=显式指定哪一列为key,当有多个key相同时使用

In [64]:
pd.merge(table3, table4, on=["手机型号","发货地区"])

,经销商,发货地区,手机型号,价格
0,dancer,beijing,iPhone,7000
1,lucy,beijing,Android,3600
2,tom,guangzhou,iPhone,7600
3,petter,shenzhen,windowsPhone,2750
4,mery,guangzhou,Android,4200


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

In [68]:
table6 = table1.copy()
table6.columns = ["型号","参考价格"]
table6

,型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


In [69]:
table2

,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


In [75]:
# 如果合并的列名称不同，会自动保留所有的原始列，可以用户自己决定删除哪一列
r1 = pd.merge(table6, table2, left_on="型号", right_on="手机型号")
r1

,型号,参考价格,手机型号,重量
0,windowsPhone,2500,windowsPhone,0.50
1,iPhone,7500,iPhone,0.40
2,Android,4000,Android,0.45


In [78]:
# 删除行或者列
r1.drop(labels=["型号"], axis=1, inplace=True)

In [79]:
r1

,参考价格,手机型号,重量
0,2500,windowsPhone,0.50
1,7500,iPhone,0.40
2,4000,Android,0.45


In [77]:
r1.drop(labels=[0])

,型号,参考价格,手机型号,重量
1,iPhone,7500,iPhone,0.40
2,Android,4000,Android,0.45


============================================

练习16：

1. 假设有两份成绩单，除了ddd是张三李四王老五之外，还有ddd4是张三和赵小六的成绩单，如何合并？

2. 如果ddd4中张三的名字被打错了，成为了张十三，怎么办？

3. 自行练习多对一，多对多的情况  

4. 自学left_index,right_index

============================================

In [109]:
# 合并只能参考列来合并
df1 = DataFrame(data={
    "张三":["A","B","C"],
    "李四":["B","B","A"],
    "王老五":["C","B","C"]
})

df2 = DataFrame(data={
    "张十三":["A","B","C"],
    "赵小六":["C","D","E"]
})
display(df1, df2)

,张三,李四,王老五
0,A,B,C
1,B,B,B
2,C,A,C


,张十三,赵小六
0,A,C
1,B,D
2,C,E


In [108]:
pd.merge(df1, df2)

,张三,李四,王老五,赵小六
0,A,B,C,C
1,B,B,B,D
2,C,A,C,E


In [111]:
pd.merge(df1, df2, left_on="张三", right_on="张十三").drop("张十三",axis=1)

,张三,李四,王老五,赵小六
0,A,B,C,C
1,B,B,B,D
2,C,A,C,E


#### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

- 外合并 how='outer'：补NaN

- 左合并、右合并：how='left'，how='right'，

In [81]:
table1

,手机型号,参考价格
0,windowsPhone,2500
1,iPhone,7500
2,Android,4000


In [82]:
table2

,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


In [80]:
# inner 内合并，只保留相同的【内容】
# concat 的内连接： 只保留相同的【标签】
pd.merge(table1, table2, how='inner')

,手机型号,参考价格,重量
0,windowsPhone,2500,0.50
1,iPhone,7500,0.40
2,Android,4000,0.45


In [83]:
# 保留合并列内容的并集
pd.merge(table1, table2, how='outer')

,手机型号,参考价格,重量
0,windowsPhone,2500.0,0.50
1,iPhone,7500.0,0.40
2,Android,4000.0,0.45
3,other,NaN,0.60


In [86]:
# left\right  根据左表和右表来保留内容
pd.merge(table1, table2, how='left')
pd.merge(table1, table2, how='right')

,手机型号,参考价格,重量
0,windowsPhone,2500.0,0.50
1,iPhone,7500.0,0.40
2,Android,4000.0,0.45
3,other,NaN,0.60


============================================

练习17：



1. 考虑应用情景，使用多种方式合并ddd与ddd4

============================================

#### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

In [87]:
table3

,经销商,发货地区,手机型号
0,dancer,beijing,iPhone
1,lucy,beijing,Android
2,tom,guangzhou,iPhone
3,petter,shenzhen,windowsPhone
4,mery,guangzhou,Android


In [88]:
table4

,发货地区,手机型号,价格
0,beijing,iPhone,7000
1,beijing,windowsPhone,2300
2,beijing,Android,3600
3,guangzhou,iPhone,7600
4,guangzhou,windowsPhone,2800
5,guangzhou,Android,4200
6,shenzhen,iPhone,7400
7,shenzhen,windowsPhone,2750
8,shenzhen,Android,3900


In [91]:
pd.merge(table3, table4, on="手机型号", suffixes=["_下半年", "_上半年"])

,经销商,发货地区_下半年,手机型号,发货地区_上半年,价格
0,dancer,beijing,iPhone,beijing,7000
1,dancer,beijing,iPhone,guangzhou,7600
2,dancer,beijing,iPhone,shenzhen,7400
3,tom,guangzhou,iPhone,beijing,7000
4,tom,guangzhou,iPhone,guangzhou,7600
5,tom,guangzhou,iPhone,shenzhen,7400
6,lucy,beijing,Android,beijing,3600
7,lucy,beijing,Android,guangzhou,4200
8,lucy,beijing,Android,shenzhen,3900
9,mery,guangzhou,Android,beijing,3600


============================================

练习18：

    假设有两个同学都叫李四，ddd5、ddd6都是张三和李四的成绩表，如何合并？

============================================

In [112]:
# 合并只能参考列来合并
df1 = DataFrame(data={
    "张三":["A","B","C"],
    "李四":["B","B","A"],
    "王老五":["C","B","C"]
})

df2 = DataFrame(data={
    "张三":["A","B","C"],
    "李四":["C","D","E"]
})
display(df1, df2)

,张三,李四,王老五
0,A,B,C
1,B,B,B
2,C,A,C


,张三,李四
0,A,C
1,B,D
2,C,E


In [115]:
pd.merge(df1, df2, on="张三", suffixes=["_大","_小"])

,张三,李四_大,王老五,李四_小
0,A,B,C,C
1,B,B,B,D
2,C,A,C,E


pd.merge()
1. 都是以列为合并项
2. 参与合并的列必须满足一对一、一对多、多对多关系中的至少一种
3. 参与合并的列的选择应该是选择离散型数据而不是连续型数据

---
- how='inner'  合并的方式  内合并 外合并 左合并 右合并
- on  指定参与合并的列 用于有多列标签相同的情况
- left_on\right_on 分别制定左右表参与合并的列 用于两张表参与合并的列标签不同的情况
- suffiexes 一般与on参数一起使用，给相同列标签但是没有参与合并的列添加后缀
- left_index\right_index 指定索引作为合并的参考值

In [93]:
df1 = DataFrame(data={
    "age":[18,19,20,18],
    "java":[56,58,50,60],
    "name":["lucy","tom","jack","mery"]
})

df2 = DataFrame(data={
    "python":[56,59,58,56],
    "java":[56,76,60,60]
})

In [94]:
display(df1, df2)

,age,java,name
0,18,56,lucy
1,19,58,tom
2,20,50,jack
3,18,60,mery


,java,python
0,56,56
1,76,59
2,60,58
3,60,56


In [95]:
pd.merge(df1, df2)

,age,java,name,python
0,18,56,lucy,56
1,18,60,mery,58
2,18,60,mery,56


In [100]:
# 把手机型号列设置为行索引的方法
table7 = table1.set_index("手机型号")

In [101]:
table7

,参考价格
手机型号,
windowsPhone,2500
iPhone,7500
Android,4000


In [102]:
table2

,手机型号,重量
0,windowsPhone,0.50
1,iPhone,0.40
2,Android,0.45
3,other,0.60


In [104]:
pd.merge(table7, table2, left_index=True, right_on="手机型号")

,参考价格,手机型号,重量
0,2500,windowsPhone,0.50
1,7500,iPhone,0.40
2,4000,Android,0.45


In [106]:
pd.merge(table2, table7, left_on="手机型号", right_index=True)

,手机型号,重量,参考价格
0,windowsPhone,0.50,2500
1,iPhone,0.40,7500
2,Android,0.45,4000


## 作业
## 3. 案例分析：美国各州人口数据分析

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

首先导入文件，并查看数据样本

In [8]:
abb = pd.read_csv('state-abbrevs.csv')
abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [6]:
area = pd.read_csv('state-areas.csv')
area.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [7]:
pop = pd.read_csv('state-population.csv')
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [11]:
# eg. 如何查看有重复项的列中出现了哪些值？？
pop.year.unique()

array([2012, 2010, 2011, 2009, 2013, 2007, 2008, 2005, 2006, 2004, 2003,
       2001, 2002, 1999, 2000, 1998, 1997, 1996, 1995, 1994, 1993, 1992,
       1991, 1990], dtype=int64)

In [9]:
# 先查看abb表里面有多少个州的映射
abb.shape

(51, 2)

In [13]:
# 说明abb表提供了51个州的全称和缩写的映射
abb.abbreviation.unique().size

51

In [14]:
# 查看映射表中是否存在空值
abb.isnull().any()

state           False
abbreviation    False
dtype: bool

In [16]:
# 因为我们abb表最终要和人口表pop还有面积表area进行合并，所以先查看pop\area两张表中出现的州的个数是否匹配
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [19]:
# 查看pop里面州的个数, 发现人口表中出现的州比abb表多2个
pop["state/region"].unique().size

53

In [21]:
# 查看area表里面州的个数, 发现面积表中出现的州比abb表多1个
area["state"].unique().size

52

合并pop与abbrevs两个DataFrame，分别依据state/region列和abbreviation列来合并。

为了保留所有信息，使用外合并。

In [24]:
pop.head(1)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0


In [25]:
abb.head(1)

,state,abbreviation
0,Alabama,AL


In [27]:
pop_abb = pd.merge(pop, abb, how='outer', left_on="state/region", right_on="abbreviation")

去除abbreviation的那一列（axis=1）

In [28]:
pop_abb

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
5,AL,total,2011,4801627.0,Alabama,AL
6,AL,total,2009,4757938.0,Alabama,AL
7,AL,under18,2009,1134192.0,Alabama,AL
8,AL,under18,2013,1111481.0,Alabama,AL
9,AL,total,2013,4833722.0,Alabama,AL


查看存在缺失数据的列。

使用.isnull().any()，只有某一列存在一个缺失数据，就会显示True。

In [33]:
# 找到那些州只统计了人口，但是没有全称和缩写的映射
# 只需要找到state这一列中有缺失值的行
# 这个BOOL列表，返回True的说明对应的行有缺失值
bool_list = pop_abb.isnull().any(axis=1)
# 使用上面的BOOL列表过滤总表
temp = pop_abb.loc[bool_list]

In [36]:
# state\abbreviation 返回的是True,说明这两列都为空
temp.isnull().all()

state/region    False
ages            False
year            False
population      False
state            True
abbreviation     True
dtype: bool

In [39]:
# 查看哪些州没有全称
temp["state/region"].unique()

array(['PR', 'USA'], dtype=object)

In [41]:
# 可以把PR\USA的全程补充到pop_abb这个总表中
pop_abb

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
5,AL,total,2011,4801627.0,Alabama,AL
6,AL,total,2009,4757938.0,Alabama,AL
7,AL,under18,2009,1134192.0,Alabama,AL
8,AL,under18,2013,1111481.0,Alabama,AL
9,AL,total,2013,4833722.0,Alabama,AL


In [47]:
# 填充PR州的全称
# df.loc[index, column]
# df.loc[bool_list, column]
pop_abb.loc[pop_abb["state/region"] == "PR", "state"] = "Puerto Rico"

In [48]:
# 填充USA的全称
pop_abb.loc[pop_abb["state/region"] == "USA", "state"] = "USA"

In [40]:
area

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [50]:
# 查看缺失数据，现在人口有缺失，abbreviation有缺失，但是state/region和abbreviation是一个字段，并且state/region没有缺失值，所以可以删除abbreviation列
pop_abb.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
abbreviation     True
dtype: bool

In [51]:
pop_abb.drop("abbreviation", axis=1, inplace=True)

查看缺失数据

In [52]:
pop_abb.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

根据数据是否缺失情况显示数据，如果缺失为True，那么显示

找到有哪些state/region使得state的值为NaN，使用unique()查看非重复值

为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN！

记住这样清除缺失数据NaN的方法！

合并各州面积数据areas，使用左合并。

思考一下为什么使用外合并？



In [59]:
pop_abb["state"].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico', 'USA'],
      dtype=object)

In [60]:
area["state"].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming',
       'District of Columbia', 'Puerto Rico'], dtype=object)

In [63]:
pop_abb.head(1)

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama


In [64]:
area.head(1)

,state,area (sq. mi)
0,Alabama,52423


In [62]:
# 查看哪个州是多出来的, USA是多出来的
set(pop_abb["state"].unique()) - set(area["state"].unique())

{'USA'}

In [65]:
# 合并总表
total = pd.merge(pop_abb, area, how='outer')

继续寻找存在缺失数据的列

In [67]:
total.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

我们会发现area(sq.mi)这一列有缺失数据，为了找出是哪一行，我们需要找出是哪个state没有数据

In [71]:
# 发现只有USA是存在面积缺失的，但是USA是整个美国的数据，所以不涉及州的人口密度，所以可以删除
# df.loc[bool_list]
total.loc[total["area (sq. mi)"].isnull()]["state/region"].unique()

array(['USA'], dtype=object)

去除含有缺失数据的行

In [ ]:
# 删除的逻辑
total.drop([所有面积为空的行标签])
# 过滤的逻辑
total.loc[所有面积非空的行索引列表]

In [75]:
# 第一种办法：获取面积为空的行索引
emtpy_indexes = total.loc[total["area (sq. mi)"].isnull()].index
total1 = total.drop(emtpy_indexes)

In [76]:
total1.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)    False
dtype: bool

In [78]:
# 第二种办法：获取所有面积非空的行
not_empty_indexes = total["area (sq. mi)"].notnull()
total2 = total.loc[not_empty_indexes]

In [79]:
total2.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)    False
dtype: bool

查看数据是否缺失

In [81]:
# 继续查看哪些州有人口数据的缺失
total2.loc[total2["population"].isnull()]

,state/region,ages,year,population,state,area (sq. mi)
2448,PR,under18,1990,NaN,Puerto Rico,3515.0
2449,PR,total,1990,NaN,Puerto Rico,3515.0
2450,PR,total,1991,NaN,Puerto Rico,3515.0
2451,PR,under18,1991,NaN,Puerto Rico,3515.0
2452,PR,total,1993,NaN,Puerto Rico,3515.0
2453,PR,under18,1993,NaN,Puerto Rico,3515.0
2454,PR,under18,1992,NaN,Puerto Rico,3515.0
2455,PR,total,1992,NaN,Puerto Rico,3515.0
2456,PR,under18,1994,NaN,Puerto Rico,3515.0
2457,PR,total,1994,NaN,Puerto Rico,3515.0


In [84]:
# 因为人口数据官方没有统计，所以删除
# 这里采用过滤population非空的方式获取结果
result = total2.loc[total2["population"].notnull()]

In [85]:
result.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [86]:
result.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


找出2010年的全民人口数据,df.query(查询语句)

In [ ]:
# year == 2010
# ages = "total"

In [90]:
condition = (result["year"] == 2010) & (result["ages"] == "total")
total_2010 = result.loc[condition]

In [92]:
total_2010_a = result.query("year == 2010 & ages == 'total'")

对查询结果进行处理，以state列作为新的行索引:set_index

In [94]:
total_2010["population"]/total_2010["area (sq. mi)"]

3         91.287603
91         1.087509
101       56.214497
189       54.948667
197      228.051342
283       48.493718
293      645.600649
379      460.445752
389     8898.897059
475      286.597129
485      163.409902
570      124.746707
581       18.794338
666      221.687472
677      178.197831
762       54.202751
773       34.745266
858      107.586994
869       87.676099
954       37.509990
965      466.445797
1050     621.815538
1061     102.015794
1146      61.078373
1157      61.321530
1242      86.015622
1253       6.736171
1338      23.654153
1349      24.448796
1434     140.799273
1445    1009.253268
1530      16.982737
1541     356.094135
1626     177.617157
1637       9.537565
1722     257.549634
1733      53.778278
1818      39.001565
1829     275.966651
1914     681.339159
1925     144.854594
2010      10.583512
2021     150.825298
2106      93.987655
2117      32.677188
2202      65.085075
2213     187.622273
2298      94.557817
2309      76.519582
2394      86.851900


计算人口密度。注意是Series/Series，其结果还是一个Series。

In [98]:
temp1 = total_2010.set_index("state")

In [102]:
temp1["area (sq. mi)"]

state
Alabama                  52423.0
Alaska                  656425.0
Arizona                 114006.0
Arkansas                 53182.0
California              163707.0
Colorado                104100.0
Connecticut               5544.0
Delaware                  1954.0
District of Columbia        68.0
Florida                  65758.0
Georgia                  59441.0
Hawaii                   10932.0
Idaho                    83574.0
Illinois                 57918.0
Indiana                  36420.0
Iowa                     56276.0
Kansas                   82282.0
Kentucky                 40411.0
Louisiana                51843.0
Maine                    35387.0
Maryland                 12407.0
Massachusetts            10555.0
Michigan                 96810.0
Minnesota                86943.0
Mississippi              48434.0
Missouri                 69709.0
Montana                 147046.0
Nebraska                 77358.0
Nevada                  110567.0
New Hampshire             9351.0
New 

In [103]:
density = temp1["population"]/temp1["area (sq. mi)"]

In [104]:
density

state
Alabama                   91.287603
Alaska                     1.087509
Arizona                   56.214497
Arkansas                  54.948667
California               228.051342
Colorado                  48.493718
Connecticut              645.600649
Delaware                 460.445752
District of Columbia    8898.897059
Florida                  286.597129
Georgia                  163.409902
Hawaii                   124.746707
Idaho                     18.794338
Illinois                 221.687472
Indiana                  178.197831
Iowa                      54.202751
Kansas                    34.745266
Kentucky                 107.586994
Louisiana                 87.676099
Maine                     37.509990
Maryland                 466.445797
Massachusetts            621.815538
Michigan                 102.015794
Minnesota                 61.078373
Mississippi               61.321530
Missouri                  86.015622
Montana                    6.736171
Nebraska              

排序，并找出人口密度最高的五个州sort_values()

In [106]:
density.sort_values(ascending=False)[:5]

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

找出人口密度最低的五个州

In [107]:
density.sort_values(ascending=False)[-5:]

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

要点总结：
- 统一用loc()索引
- 善于使用.isnull().any()找到存在NaN的列
- 善于使用.unique()确定该列中哪些key是我们需要的
- 一般使用外合并、左合并，目的只有一个：宁愿该列是NaN也不要丢弃其他列的信息

## 回顾：Series/DataFrame运算与ndarray运算的区别

- Series与DataFrame没有广播，如果对应index没有值，则记为NaN；或者使用add的fill_value来补缺失值
- ndarray有广播，通过重复已有值来计算